In [1]:
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import time

import pathlib
import json
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
def load_head_data(experiments_path):
    head_data = {}
    for task_dir in experiments_path.iterdir():
        head_data[task_dir.stem] = {}
        for seed_dir in task_dir.iterdir():
            head_mask = np.load(seed_dir / "head_mask.npy")
            head_data[task_dir.stem][seed_dir.stem] = {
                "head_mask": head_mask,
            }
    return head_data
def load_mlp_data(experiments_path):
    mlp_data = {}
    for task_dir in experiments_path.iterdir():
        mlp_data[task_dir.stem] = {}
        for seed_dir in task_dir.iterdir():
            mlp_mask = np.load(seed_dir / "mlp_mask.npy")
            mlp_importance = np.load(seed_dir / "mlp_importance.npy")
            mlp_data[task_dir.stem][seed_dir.stem] = {
                "mlp_mask": mlp_mask,
                "mlp_importance": mlp_importance
            }
    return mlp_data
def plot_all_task_metrics(metrics, title):
    for name in metrics[0][1].keys():
        plot_task_metrics(analyzed_metrics, name, title)


In [3]:
experiments_path = pathlib.Path("../masks/heads_mlps")
together_head_data = load_head_data(experiments_path)
experiments_path = pathlib.Path("../masks/heads_mlps")
together_mlp_data = load_mlp_data(experiments_path)

In [4]:
super_save_path = pathlib.Path("../masks/heads_mlps_super")
super_bad_save_path = pathlib.Path("../masks/heads_mlps_super_bad")
super_bad_save_path = pathlib.Path("../masks/heads_mlps_super_random")

In [73]:
import random
random.seed(1337)



def sample_bad_to_similar_size(bad, good):
    total_good = good.sum()
    total_bad = bad.sum()
    if total_good > total_bad:
        to_add = total_good - total_bad
        
        selection = np.argwhere((good == 0) & (bad == 0)).tolist()
        if len(selection) < to_add:
            selection = np.argwhere(bad == 0).tolist()
        
        bad_indices = random.sample(selection, int(to_add))
        for idx in bad_indices:
            if len(idx) == 2:
                bad[idx[0], idx[1]] = 1
            else:
                bad[idx[0]] = 1
    elif total_good < total_bad:
        to_remove = total_bad - total_good
        bad_indices = random.sample(np.argwhere(bad == 1).tolist(), int(to_remove))
        for idx in bad_indices:
            if len(idx) == 2:
                bad[idx[0], idx[1]] = 0
            else:
                bad[idx[0]] = 0
    return bad



super_survivior_ids = {}


for task in sorted(together_head_data.keys()):
    print(f"{task}")

    super_head = np.ones((12, 12))
    super_mlp = np.ones((12,))
    super_bad_head = np.ones((12, 12))
    super_bad_mlp = np.ones((12,))
    for seed in together_head_data[task]:
        super_head *= together_head_data[task][seed]["head_mask"]
        super_mlp *= together_mlp_data[task][seed]["mlp_mask"]
        super_bad_head *= (1 - together_head_data[task][seed]["head_mask"])
        super_bad_mlp *= (1 - together_mlp_data[task][seed]["mlp_mask"])
    super_survivior_ids[task] = {
        "head": [x[0] for x in np.argwhere(super_head.reshape(-1) == 1)],
        "mlp": [x[0] for x in np.argwhere(super_mlp.reshape(-1) == 1)]
    }
    if super_head.sum() != 0:
        ultra_head *= super_head
    if super_mlp.sum() != 0:
        ultra_mlp *= super_mlp
    
    super_random_head =
    super_random_mlp = 
    
    super_bad_head = sample_bad_to_similar_size(super_bad_head, super_head)
    super_bad_mlp = sample_bad_to_similar_size(super_bad_mlp, super_mlp)
    
    print(f"Super heads: {super_head.sum()}")
    print(f"Super MLPs:  {super_mlp.sum()}")
#     print(super_bad_head.sum(), super_head.sum(), (super_bad_head * super_head).sum())
#     print(super_bad_mlp.sum(), super_mlp.sum(), (super_bad_mlp * super_mlp).sum())
#     !mkdir -p ../masks/heads_mlps_super/$task
#     !mkdir -p ../masks/heads_mlps_super_bad/$task
        
#     for seed in together_head_data[task]:
#         !mkdir -p ../masks/heads_mlps_super/$task/$seed
#         !mkdir -p ../masks/heads_mlps_super_bad/$task/$seed
#         np.save(str(super_save_path / f"{task}/{seed}/head_mask.npy"), super_head)
#         np.save(str(super_save_path / f"{task}/{seed}/mlp_mask.npy"), super_mlp)
#         np.save(str(super_bad_save_path / f"{task}/{seed}/head_mask.npy"), super_bad_head)
#         np.save(str(super_bad_save_path / f"{task}/{seed}/mlp_mask.npy"), super_bad_mlp)

CoLA
Super heads: 34.0
Super MLPs:  9.0
MNLI
Super heads: 40.0
Super MLPs:  4.0
MRPC
Super heads: 28.0
Super MLPs:  4.0
QNLI
Super heads: 31.0
Super MLPs:  5.0
QQP
Super heads: 23.0
Super MLPs:  3.0
RTE
Super heads: 11.0
Super MLPs:  2.0
SST-2
Super heads: 19.0
Super MLPs:  2.0
STS-B
Super heads: 24.0
Super MLPs:  1.0
WNLI
Super heads: 0.0
Super MLPs:  0.0


In [71]:
from itertools import combinations

for task_1, task_2 in combinations(sorted(super_survivior_ids.keys()), 2):
    print(f"{task_1}, {task_2}")
    print("--------------------")
    overlap_head = set(super_survivior_ids[task_1]["head"]) & set(super_survivior_ids[task_2]["head"])
    overlap_mlp = set(super_survivior_ids[task_1]["mlp"]) & set(super_survivior_ids[task_2]["mlp"])

    print(f"Overlaping Super Heads - {len(overlap_head)}")
    print(f"Overlaping Super MLPs - {len(overlap_mlp)}")

CoLA, MNLI
--------------------
Overlaping Super Heads - 10
Overlaping Super MLPs - 4
CoLA, MRPC
--------------------
Overlaping Super Heads - 9
Overlaping Super MLPs - 3
CoLA, QNLI
--------------------
Overlaping Super Heads - 12
Overlaping Super MLPs - 5
CoLA, QQP
--------------------
Overlaping Super Heads - 8
Overlaping Super MLPs - 3
CoLA, RTE
--------------------
Overlaping Super Heads - 3
Overlaping Super MLPs - 2
CoLA, SST-2
--------------------
Overlaping Super Heads - 6
Overlaping Super MLPs - 2
CoLA, STS-B
--------------------
Overlaping Super Heads - 10
Overlaping Super MLPs - 1
CoLA, WNLI
--------------------
Overlaping Super Heads - 0
Overlaping Super MLPs - 0
MNLI, MRPC
--------------------
Overlaping Super Heads - 17
Overlaping Super MLPs - 1
MNLI, QNLI
--------------------
Overlaping Super Heads - 14
Overlaping Super MLPs - 2
MNLI, QQP
--------------------
Overlaping Super Heads - 11
Overlaping Super MLPs - 1
MNLI, RTE
--------------------
Overlaping Super Heads - 7
Ov